# Summary:
- No SMOTE
- Based on the 32x32x3 size vectors
- Below networks are tested with below results:


    - SVM 

                         precision    recall  f1-score   support

        0.0               0.88      0.85      0.86      2666
        1.0               0.81      0.85      0.83      2112

        avg / total       0.85      0.85      0.85      4778

        Confusion Matrix
            2255  411
             313 1799


    - DNN + SVM

                     precision    recall  f1-score   support

        0.0               0.88      0.84      0.86      2666
        1.0               0.81      0.86      0.83      2112

        avg / total       0.85      0.85      0.85      4778

        Confusion Matrix
            2242  424
             295 1817
             
               
    - Deeper DNN with dropout + SVM

                     precision    recall  f1-score   support

        0.0               0.89      0.83      0.86      2666
        1.0               0.81      0.87      0.84      2112

        avg / total       0.85      0.85      0.85      4778

        Confusion Matrix
            2225  441
             274 1838
    
    - Inception Batch Normalization + SVM

                     precision    recall  f1-score   support

        0.0               0.86      0.78      0.82      2666
        1.0               0.75      0.84      0.79      2112

        avg / total       0.81      0.81      0.81      4778

        Confusion Matrix
            2081  585
             341 1771


In [1]:
import glob
import multiprocessing
import time
import csv
import pickle
import os
import pandas as pd
import shutil
import numpy as np
import warnings
import tqdm
import logging
import mxnet as mx
from bs4 import BeautifulSoup
from urllib.request import urlopen
from os.path import basename
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

%pylab inline
warnings.filterwarnings("ignore")

/home/dc/anaconda3/envs/mxnet/lib/python3.6/site-packages/scipy/sparse/sparsetools.py:20: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


Populating the interactive namespace from numpy and matplotlib


In [2]:
# 2d to 4d conversion
def to4d(img):
    return img.reshape(img.shape[0], 3, 32, 32).astype(np.float32)

In [3]:
def modelMetric(mod, info, nd=True):
    if nd == True: 
        prob = mod.predict(val_iters).asnumpy()
    else:
        prob = mod.predict(val_iters)
    logging.info(info)
    # Check the accuracy from prediction
    val_iters.reset()

    y_batch = []
    for dbatch in val_iters:
        label = dbatch.label[0].asnumpy()
        pad = val_iters.getpad()
        real_size = label.shape[0] - pad
        y_batch.append(label[0:real_size])
    y = np.concatenate(y_batch)

    # get prediction label 
    py = np.argmax(prob, axis=1)
    acc1 = sum(py == y) / len(y)
    logging.info('Test accuracy = %f', acc1)
    cfn_matrix=confusion_matrix(y,py)
    print(classification_report (y,py))
    print(cfn_matrix)

# 1. Data Preparation

In [4]:
p_file = 'RCT_Vectors'
with open(p_file, 'rb') as fin:
    vectors = pickle.load(fin)

In [5]:
p_file = 'RCT_labels'
with open(p_file, 'rb') as fin:
    data_Y = pickle.load(fin)

In [6]:
d2v_X_train, d2v_X_test, d2v_Y_train, d2v_Y_test = train_test_split(vectors, data_Y, test_size=0.25)

In [7]:
X_train_array = np.array(d2v_X_train)
Y_train_array = np.array(d2v_Y_train)
X_test_array = np.array(d2v_X_test)
Y_test_array = np.array(d2v_Y_test)

In [8]:
# double check whether oversampling works
X_train_array.shape, Y_train_array.shape, X_test_array.shape, Y_test_array.shape, len(vectors)

((14333, 3072), (14333,), (4778, 3072), (4778,), 19111)

In [9]:
# creating 4d training/testing dataset
batch_size = 64
train_iters = mx.io.NDArrayIter(to4d(X_train_array), Y_train_array, batch_size, shuffle=True,label_name='svm_label')
val_iters = mx.io.NDArrayIter(to4d(X_test_array), Y_test_array, batch_size,label_name='svm_label')

# 2. Train models
## 2.1 SVM

In [10]:
# Single SVM
data = mx.symbol.Variable('data')
svm  = mx.symbol.FullyConnected(data = data, name='fc', num_hidden=2)

# Here we add the ultimate layer based on L2-SVM objective
ssvm = mx.symbol.SVMOutput(data=svm, name='svm')
# draw network chart
shape = {"data" : (batch_size, 3, 32, 32)}
mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=ssvm, shape=shape)
network_chart.format = 'png'
network_chart.render('SVM')

'SVM.png'

<img src='SVM.png'>

In [11]:
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

mod = mx.model.FeedForward(
    ctx = mx.gpu(0),     # use GPU 0 for training, others are same as before
    symbol = ssvm,       # network structure
    num_epoch = 300,     # number of data passes for training 
    learning_rate = 0.00001 # learning rate of SGD 
)
mod.fit(
    X=train_iters,       # training data
    eval_data=val_iters, # validation data
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size) # output progress for each 200 data batches
)


INFO:root:Start training with [gpu(0)]
INFO:root:Epoch[0] Batch [64]	Speed: 42339.66 samples/sec	accuracy=0.507324
INFO:root:Epoch[0] Batch [128]	Speed: 42547.60 samples/sec	accuracy=0.534424
INFO:root:Epoch[0] Batch [192]	Speed: 41729.20 samples/sec	accuracy=0.533203
INFO:root:Epoch[0] Resetting Data Iterator
INFO:root:Epoch[0] Time cost=0.355
INFO:root:Epoch[0] Validation-accuracy=0.547292
INFO:root:Epoch[1] Batch [64]	Speed: 49067.12 samples/sec	accuracy=0.556641
INFO:root:Epoch[1] Batch [128]	Speed: 53930.45 samples/sec	accuracy=0.583984
INFO:root:Epoch[1] Batch [192]	Speed: 56838.43 samples/sec	accuracy=0.579346
INFO:root:Epoch[1] Resetting Data Iterator
INFO:root:Epoch[1] Time cost=0.272
INFO:root:Epoch[1] Validation-accuracy=0.595208
INFO:root:Epoch[2] Batch [64]	Speed: 51709.37 samples/sec	accuracy=0.603271
INFO:root:Epoch[2] Batch [128]	Speed: 54875.84 samples/sec	accuracy=0.625244
INFO:root:Epoch[2] Batch [192]	Speed: 53432.78 samples/sec	accuracy=0.621094
INFO:root:Epoch[2] 

INFO:root:Epoch[22] Time cost=0.259
INFO:root:Epoch[22] Validation-accuracy=0.804583
INFO:root:Epoch[23] Batch [64]	Speed: 54317.17 samples/sec	accuracy=0.806885
INFO:root:Epoch[23] Batch [128]	Speed: 52853.49 samples/sec	accuracy=0.806641
INFO:root:Epoch[23] Batch [192]	Speed: 52539.27 samples/sec	accuracy=0.802246
INFO:root:Epoch[23] Resetting Data Iterator
INFO:root:Epoch[23] Time cost=0.276
INFO:root:Epoch[23] Validation-accuracy=0.806250
INFO:root:Epoch[24] Batch [64]	Speed: 51654.95 samples/sec	accuracy=0.808105
INFO:root:Epoch[24] Batch [128]	Speed: 49582.73 samples/sec	accuracy=0.808594
INFO:root:Epoch[24] Batch [192]	Speed: 56696.06 samples/sec	accuracy=0.804932
INFO:root:Epoch[24] Resetting Data Iterator
INFO:root:Epoch[24] Time cost=0.276
INFO:root:Epoch[24] Validation-accuracy=0.807708
INFO:root:Epoch[25] Batch [64]	Speed: 54924.26 samples/sec	accuracy=0.810547
INFO:root:Epoch[25] Batch [128]	Speed: 57759.69 samples/sec	accuracy=0.810059
INFO:root:Epoch[25] Batch [192]	Spee

INFO:root:Epoch[45] Batch [192]	Speed: 43687.89 samples/sec	accuracy=0.828369
INFO:root:Epoch[45] Resetting Data Iterator
INFO:root:Epoch[45] Time cost=0.332
INFO:root:Epoch[45] Validation-accuracy=0.831667
INFO:root:Epoch[46] Batch [64]	Speed: 55829.01 samples/sec	accuracy=0.838867
INFO:root:Epoch[46] Batch [128]	Speed: 54800.40 samples/sec	accuracy=0.836914
INFO:root:Epoch[46] Batch [192]	Speed: 44211.24 samples/sec	accuracy=0.829346
INFO:root:Epoch[46] Resetting Data Iterator
INFO:root:Epoch[46] Time cost=0.296
INFO:root:Epoch[46] Validation-accuracy=0.831667
INFO:root:Epoch[47] Batch [64]	Speed: 54461.47 samples/sec	accuracy=0.839111
INFO:root:Epoch[47] Batch [128]	Speed: 51452.91 samples/sec	accuracy=0.837402
INFO:root:Epoch[47] Batch [192]	Speed: 44911.87 samples/sec	accuracy=0.830322
INFO:root:Epoch[47] Resetting Data Iterator
INFO:root:Epoch[47] Time cost=0.297
INFO:root:Epoch[47] Validation-accuracy=0.832917
INFO:root:Epoch[48] Batch [64]	Speed: 54451.28 samples/sec	accuracy=0

INFO:root:Epoch[68] Batch [128]	Speed: 52609.26 samples/sec	accuracy=0.848389
INFO:root:Epoch[68] Batch [192]	Speed: 52594.28 samples/sec	accuracy=0.839111
INFO:root:Epoch[68] Resetting Data Iterator
INFO:root:Epoch[68] Time cost=0.278
INFO:root:Epoch[68] Validation-accuracy=0.841250
INFO:root:Epoch[69] Batch [64]	Speed: 57453.53 samples/sec	accuracy=0.851318
INFO:root:Epoch[69] Batch [128]	Speed: 51328.85 samples/sec	accuracy=0.848877
INFO:root:Epoch[69] Batch [192]	Speed: 51837.47 samples/sec	accuracy=0.838623
INFO:root:Epoch[69] Resetting Data Iterator
INFO:root:Epoch[69] Time cost=0.274
INFO:root:Epoch[69] Validation-accuracy=0.841042
INFO:root:Epoch[70] Batch [64]	Speed: 55027.00 samples/sec	accuracy=0.851807
INFO:root:Epoch[70] Batch [128]	Speed: 55392.31 samples/sec	accuracy=0.849121
INFO:root:Epoch[70] Batch [192]	Speed: 51305.70 samples/sec	accuracy=0.838623
INFO:root:Epoch[70] Resetting Data Iterator
INFO:root:Epoch[70] Time cost=0.271
INFO:root:Epoch[70] Validation-accuracy=

INFO:root:Epoch[90] Validation-accuracy=0.846667
INFO:root:Epoch[91] Batch [64]	Speed: 53160.31 samples/sec	accuracy=0.858398
INFO:root:Epoch[91] Batch [128]	Speed: 53642.50 samples/sec	accuracy=0.854980
INFO:root:Epoch[91] Batch [192]	Speed: 51905.44 samples/sec	accuracy=0.845703
INFO:root:Epoch[91] Resetting Data Iterator
INFO:root:Epoch[91] Time cost=0.279
INFO:root:Epoch[91] Validation-accuracy=0.846875
INFO:root:Epoch[92] Batch [64]	Speed: 54164.93 samples/sec	accuracy=0.858398
INFO:root:Epoch[92] Batch [128]	Speed: 51221.42 samples/sec	accuracy=0.854980
INFO:root:Epoch[92] Batch [192]	Speed: 56201.40 samples/sec	accuracy=0.845459
INFO:root:Epoch[92] Resetting Data Iterator
INFO:root:Epoch[92] Time cost=0.272
INFO:root:Epoch[92] Validation-accuracy=0.847500
INFO:root:Epoch[93] Batch [64]	Speed: 53520.84 samples/sec	accuracy=0.858887
INFO:root:Epoch[93] Batch [128]	Speed: 51893.05 samples/sec	accuracy=0.856201
INFO:root:Epoch[93] Batch [192]	Speed: 51520.96 samples/sec	accuracy=0.8

...

INFO:root:Epoch[203] Batch [64]	Speed: 57595.48 samples/sec	accuracy=0.874268
INFO:root:Epoch[203] Batch [128]	Speed: 63303.48 samples/sec	accuracy=0.869873
INFO:root:Epoch[203] Batch [192]	Speed: 63142.02 samples/sec	accuracy=0.859375
INFO:root:Epoch[203] Resetting Data Iterator
INFO:root:Epoch[203] Time cost=0.241
INFO:root:Epoch[203] Validation-accuracy=0.851250
INFO:root:Epoch[204] Batch [64]	Speed: 61051.85 samples/sec	accuracy=0.874268
INFO:root:Epoch[204] Batch [128]	Speed: 55096.88 samples/sec	accuracy=0.869873
INFO:root:Epoch[204] Batch [192]	Speed: 58439.50 samples/sec	accuracy=0.859131
INFO:root:Epoch[204] Resetting Data Iterator
INFO:root:Epoch[204] Time cost=0.252
INFO:root:Epoch[204] Validation-accuracy=0.851250
INFO:root:Epoch[205] Batch [64]	Speed: 60720.84 samples/sec	accuracy=0.874512
INFO:root:Epoch[205] Batch [128]	Speed: 58067.17 samples/sec	accuracy=0.869873
INFO:root:Epoch[205] Batch [192]	Speed: 59395.56 samples/sec	accuracy=0.859131
INFO:root:Epoch[205] Resetti

INFO:root:Epoch[225] Batch [192]	Speed: 56094.78 samples/sec	accuracy=0.858887
INFO:root:Epoch[225] Resetting Data Iterator
INFO:root:Epoch[225] Time cost=0.262
INFO:root:Epoch[225] Validation-accuracy=0.850417
INFO:root:Epoch[226] Batch [64]	Speed: 57353.79 samples/sec	accuracy=0.874268
INFO:root:Epoch[226] Batch [128]	Speed: 56199.01 samples/sec	accuracy=0.870361
INFO:root:Epoch[226] Batch [192]	Speed: 53942.14 samples/sec	accuracy=0.858887
INFO:root:Epoch[226] Resetting Data Iterator
INFO:root:Epoch[226] Time cost=0.265
INFO:root:Epoch[226] Validation-accuracy=0.850625
INFO:root:Epoch[227] Batch [64]	Speed: 56391.41 samples/sec	accuracy=0.874268
INFO:root:Epoch[227] Batch [128]	Speed: 54871.11 samples/sec	accuracy=0.870117
INFO:root:Epoch[227] Batch [192]	Speed: 58345.82 samples/sec	accuracy=0.859131
INFO:root:Epoch[227] Resetting Data Iterator
INFO:root:Epoch[227] Time cost=0.258
INFO:root:Epoch[227] Validation-accuracy=0.850625
INFO:root:Epoch[228] Batch [64]	Speed: 55176.51 sampl

INFO:root:Epoch[247] Time cost=0.306
INFO:root:Epoch[247] Validation-accuracy=0.849375
INFO:root:Epoch[248] Batch [64]	Speed: 64221.41 samples/sec	accuracy=0.875000
INFO:root:Epoch[248] Batch [128]	Speed: 57489.87 samples/sec	accuracy=0.872070
INFO:root:Epoch[248] Batch [192]	Speed: 60818.43 samples/sec	accuracy=0.860596
INFO:root:Epoch[248] Resetting Data Iterator
INFO:root:Epoch[248] Time cost=0.244
INFO:root:Epoch[248] Validation-accuracy=0.849167
INFO:root:Epoch[249] Batch [64]	Speed: 52512.45 samples/sec	accuracy=0.875000
INFO:root:Epoch[249] Batch [128]	Speed: 42071.43 samples/sec	accuracy=0.872070
INFO:root:Epoch[249] Batch [192]	Speed: 48844.19 samples/sec	accuracy=0.860596
INFO:root:Epoch[249] Resetting Data Iterator
INFO:root:Epoch[249] Time cost=0.306
INFO:root:Epoch[249] Validation-accuracy=0.849167
INFO:root:Epoch[250] Batch [64]	Speed: 60122.87 samples/sec	accuracy=0.875000
INFO:root:Epoch[250] Batch [128]	Speed: 59306.78 samples/sec	accuracy=0.872070
INFO:root:Epoch[250]

INFO:root:Epoch[270] Batch [128]	Speed: 39013.24 samples/sec	accuracy=0.872559
INFO:root:Epoch[270] Batch [192]	Speed: 57332.35 samples/sec	accuracy=0.863037
INFO:root:Epoch[270] Resetting Data Iterator
INFO:root:Epoch[270] Time cost=0.316
INFO:root:Epoch[270] Validation-accuracy=0.848750
INFO:root:Epoch[271] Batch [64]	Speed: 58469.73 samples/sec	accuracy=0.876709
INFO:root:Epoch[271] Batch [128]	Speed: 58762.92 samples/sec	accuracy=0.873047
INFO:root:Epoch[271] Batch [192]	Speed: 53939.93 samples/sec	accuracy=0.863037
INFO:root:Epoch[271] Resetting Data Iterator
INFO:root:Epoch[271] Time cost=0.261
INFO:root:Epoch[271] Validation-accuracy=0.848958
INFO:root:Epoch[272] Batch [64]	Speed: 50132.10 samples/sec	accuracy=0.876709
INFO:root:Epoch[272] Batch [128]	Speed: 42583.88 samples/sec	accuracy=0.872803
INFO:root:Epoch[272] Batch [192]	Speed: 39921.99 samples/sec	accuracy=0.863037
INFO:root:Epoch[272] Resetting Data Iterator
INFO:root:Epoch[272] Time cost=0.338
INFO:root:Epoch[272] Val

INFO:root:Epoch[292] Resetting Data Iterator
INFO:root:Epoch[292] Time cost=0.254
INFO:root:Epoch[292] Validation-accuracy=0.848333
INFO:root:Epoch[293] Batch [64]	Speed: 55591.63 samples/sec	accuracy=0.877686
INFO:root:Epoch[293] Batch [128]	Speed: 61052.29 samples/sec	accuracy=0.873047
INFO:root:Epoch[293] Batch [192]	Speed: 64068.37 samples/sec	accuracy=0.863525
INFO:root:Epoch[293] Resetting Data Iterator
INFO:root:Epoch[293] Time cost=0.247
INFO:root:Epoch[293] Validation-accuracy=0.848333
INFO:root:Epoch[294] Batch [64]	Speed: 59031.87 samples/sec	accuracy=0.877686
INFO:root:Epoch[294] Batch [128]	Speed: 62495.43 samples/sec	accuracy=0.873047
INFO:root:Epoch[294] Batch [192]	Speed: 65890.41 samples/sec	accuracy=0.863525
INFO:root:Epoch[294] Resetting Data Iterator
INFO:root:Epoch[294] Time cost=0.233
INFO:root:Epoch[294] Validation-accuracy=0.848333
INFO:root:Epoch[295] Batch [64]	Speed: 54612.60 samples/sec	accuracy=0.877930
INFO:root:Epoch[295] Batch [128]	Speed: 58681.63 sampl

In [12]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'SVM'
epoch = 300
mod.save(prefix, epoch)

INFO:root:Saved checkpoint to "SVM-0300.params"


In [13]:
modelMetric(mod, 'Finish predict of one Layer SVM Network...', False)

INFO:root:Finish predict of one Layer SVM Network...
INFO:root:Test accuracy = 0.848472


             precision    recall  f1-score   support

        0.0       0.88      0.85      0.86      2666
        1.0       0.81      0.85      0.83      2112

avg / total       0.85      0.85      0.85      4778

[[2255  411]
 [ 313 1799]]


## 2.2 DNN + SVM

In [14]:
# DNN + SVM 
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=512)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 512)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")

fc3  = mx.symbol.FullyConnected(data = act2, name='fc3', num_hidden=2)

dnnsvm = mx.symbol.SVMOutput(data=fc3, name='svm')

# draw network chart
shape = {"data" : (batch_size, 3, 32, 32)}
mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart=mx.viz.plot_network(symbol=dnnsvm, shape=shape)
network_chart.format = 'png'
network_chart.render('DNN + SVM')

'DNN + SVM.png'

<img src='DNN + SVM.png'>

In [15]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = dnnsvm,        
    label_names = ['svm_label']
)

lr_sch = mx.lr_scheduler.FactorScheduler(step=1800, factor=0.9)

mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size), 
    num_epoch = 300,       # Train for 100 epochs
    optimizer = 'Adam',
    optimizer_params = {
        'learning_rate': 9e-7, # Learning rate
        'beta1': 0.9,       # Beta 1
        'beta2': 0.999,         # Beta 2
        'lr_scheduler': lr_sch #learning rate decrease schedule
    },
)
print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 36161.52 samples/sec	accuracy=0.458654
INFO:root:Epoch[0] Batch [128]	Speed: 36569.33 samples/sec	accuracy=0.512207
INFO:root:Epoch[0] Batch [192]	Speed: 39929.04 samples/sec	accuracy=0.564209
INFO:root:Epoch[0] Train-accuracy=0.610887
INFO:root:Epoch[0] Time cost=0.402
INFO:root:Epoch[0] Validation-accuracy=0.611875
INFO:root:Epoch[1] Batch [64]	Speed: 43229.37 samples/sec	accuracy=0.658413
INFO:root:Epoch[1] Batch [128]	Speed: 41946.52 samples/sec	accuracy=0.715576
INFO:root:Epoch[1] Batch [192]	Speed: 38346.12 samples/sec	accuracy=0.739990
INFO:root:Epoch[1] Train-accuracy=0.774194
INFO:root:Epoch[1] Time cost=0.355
INFO:root:Epoch[1] Validation-accuracy=0.752708
INFO:root:Epoch[2] Batch [64]	Speed: 40448.63 samples/sec	accuracy=0.771635
INFO:root:Epoch[2] Batch [128]	Speed: 45666.12 samples/sec	accuracy=0.789551
INFO:root:Epoch[2] Batch [192]	Speed: 43650.59 samples/sec	accuracy=0.790527
INFO:root:Epoch[2] Train-accuracy=0.812500
INFO:root:Epoch

INFO:root:Epoch[22] Train-accuracy=0.850806
INFO:root:Epoch[22] Time cost=0.372
INFO:root:Epoch[22] Validation-accuracy=0.838333
INFO:root:Epoch[23] Batch [64]	Speed: 38614.00 samples/sec	accuracy=0.853365
INFO:root:Epoch[23] Batch [128]	Speed: 43866.82 samples/sec	accuracy=0.843262
INFO:root:Epoch[23] Batch [192]	Speed: 41525.15 samples/sec	accuracy=0.836670
INFO:root:Epoch[23] Train-accuracy=0.851815
INFO:root:Epoch[23] Time cost=0.355
INFO:root:Epoch[23] Validation-accuracy=0.839583
INFO:root:Update[5401]: Change learning rate to 6.56100e-07
INFO:root:Epoch[24] Batch [64]	Speed: 39049.95 samples/sec	accuracy=0.854087
INFO:root:Epoch[24] Batch [128]	Speed: 44946.06 samples/sec	accuracy=0.845215
INFO:root:Epoch[24] Batch [192]	Speed: 46143.74 samples/sec	accuracy=0.838379
INFO:root:Epoch[24] Train-accuracy=0.853327
INFO:root:Epoch[24] Time cost=0.346
INFO:root:Epoch[24] Validation-accuracy=0.840625
INFO:root:Epoch[25] Batch [64]	Speed: 36498.71 samples/sec	accuracy=0.854327
INFO:root:

INFO:root:Epoch[44] Validation-accuracy=0.848333
INFO:root:Epoch[45] Batch [64]	Speed: 39380.16 samples/sec	accuracy=0.870673
INFO:root:Epoch[45] Batch [128]	Speed: 41555.08 samples/sec	accuracy=0.864990
INFO:root:Epoch[45] Batch [192]	Speed: 43004.29 samples/sec	accuracy=0.854248
INFO:root:Epoch[45] Train-accuracy=0.869456
INFO:root:Epoch[45] Time cost=0.378
INFO:root:Epoch[45] Validation-accuracy=0.848542
INFO:root:Epoch[46] Batch [64]	Speed: 24387.77 samples/sec	accuracy=0.871394
INFO:root:Epoch[46] Batch [128]	Speed: 42225.19 samples/sec	accuracy=0.864258
INFO:root:Epoch[46] Batch [192]	Speed: 46005.84 samples/sec	accuracy=0.854736
INFO:root:Epoch[46] Train-accuracy=0.870968
INFO:root:Epoch[46] Time cost=0.403
INFO:root:Epoch[46] Validation-accuracy=0.848750
INFO:root:Epoch[47] Batch [64]	Speed: 36671.75 samples/sec	accuracy=0.873077
INFO:root:Epoch[47] Batch [128]	Speed: 38691.92 samples/sec	accuracy=0.864258
INFO:root:Epoch[47] Batch [192]	Speed: 29918.41 samples/sec	accuracy=0.8

INFO:root:Epoch[67] Batch [64]	Speed: 35945.81 samples/sec	accuracy=0.879327
INFO:root:Epoch[67] Batch [128]	Speed: 44624.66 samples/sec	accuracy=0.871826
INFO:root:Epoch[67] Batch [192]	Speed: 46469.25 samples/sec	accuracy=0.859375
INFO:root:Epoch[67] Train-accuracy=0.875000
INFO:root:Epoch[67] Time cost=0.347
INFO:root:Epoch[67] Validation-accuracy=0.847500
INFO:root:Epoch[68] Batch [64]	Speed: 35346.83 samples/sec	accuracy=0.879808
INFO:root:Epoch[68] Batch [128]	Speed: 34339.68 samples/sec	accuracy=0.872314
INFO:root:Epoch[68] Batch [192]	Speed: 25928.63 samples/sec	accuracy=0.859863
INFO:root:Epoch[68] Train-accuracy=0.875504
INFO:root:Epoch[68] Time cost=0.477
INFO:root:Epoch[68] Validation-accuracy=0.847292
INFO:root:Epoch[69] Batch [64]	Speed: 36998.87 samples/sec	accuracy=0.879808
INFO:root:Epoch[69] Batch [128]	Speed: 39031.05 samples/sec	accuracy=0.872314
INFO:root:Epoch[69] Batch [192]	Speed: 27771.28 samples/sec	accuracy=0.859863
INFO:root:Epoch[69] Train-accuracy=0.876008

INFO:root:Epoch[89] Batch [128]	Speed: 43706.67 samples/sec	accuracy=0.878174
INFO:root:Epoch[89] Batch [192]	Speed: 43525.74 samples/sec	accuracy=0.862793
INFO:root:Epoch[89] Train-accuracy=0.877520
INFO:root:Epoch[89] Time cost=0.352
INFO:root:Epoch[89] Validation-accuracy=0.848542
INFO:root:Epoch[90] Batch [64]	Speed: 37460.41 samples/sec	accuracy=0.882692
INFO:root:Epoch[90] Batch [128]	Speed: 39531.94 samples/sec	accuracy=0.878418
INFO:root:Epoch[90] Batch [192]	Speed: 39690.21 samples/sec	accuracy=0.863037
INFO:root:Epoch[90] Train-accuracy=0.877520
INFO:root:Epoch[90] Time cost=0.393
INFO:root:Epoch[90] Validation-accuracy=0.848542
INFO:root:Epoch[91] Batch [64]	Speed: 35778.57 samples/sec	accuracy=0.883413
INFO:root:Epoch[91] Batch [128]	Speed: 45002.81 samples/sec	accuracy=0.878418
INFO:root:Epoch[91] Batch [192]	Speed: 44549.91 samples/sec	accuracy=0.863281
INFO:root:Epoch[91] Train-accuracy=0.877520
INFO:root:Epoch[91] Time cost=0.356
INFO:root:Epoch[91] Validation-accuracy=

...

INFO:root:Epoch[220] Train-accuracy=0.887601
INFO:root:Epoch[220] Time cost=0.345
INFO:root:Epoch[220] Validation-accuracy=0.850000
INFO:root:Epoch[221] Batch [64]	Speed: 36847.46 samples/sec	accuracy=0.890385
INFO:root:Epoch[221] Batch [128]	Speed: 46953.90 samples/sec	accuracy=0.885986
INFO:root:Epoch[221] Batch [192]	Speed: 45934.13 samples/sec	accuracy=0.874023
INFO:root:Epoch[221] Train-accuracy=0.887601
INFO:root:Epoch[221] Time cost=0.341
INFO:root:Epoch[221] Validation-accuracy=0.850000
INFO:root:Epoch[222] Batch [64]	Speed: 36410.91 samples/sec	accuracy=0.890385
INFO:root:Epoch[222] Batch [128]	Speed: 44444.68 samples/sec	accuracy=0.885986
INFO:root:Epoch[222] Batch [192]	Speed: 42928.53 samples/sec	accuracy=0.874023
INFO:root:Epoch[222] Train-accuracy=0.887601
INFO:root:Epoch[222] Time cost=0.352
INFO:root:Epoch[222] Validation-accuracy=0.850000
INFO:root:Epoch[223] Batch [64]	Speed: 36052.93 samples/sec	accuracy=0.890385
INFO:root:Epoch[223] Batch [128]	Speed: 46546.81 sampl

INFO:root:Epoch[242] Batch [192]	Speed: 41857.81 samples/sec	accuracy=0.874512
INFO:root:Epoch[242] Train-accuracy=0.887601
INFO:root:Epoch[242] Time cost=0.364
INFO:root:Epoch[242] Validation-accuracy=0.850000
INFO:root:Epoch[243] Batch [64]	Speed: 27017.26 samples/sec	accuracy=0.891346
INFO:root:Epoch[243] Batch [128]	Speed: 33124.59 samples/sec	accuracy=0.885986
INFO:root:Epoch[243] Batch [192]	Speed: 43941.09 samples/sec	accuracy=0.874512
INFO:root:Epoch[243] Train-accuracy=0.887601
INFO:root:Epoch[243] Time cost=0.420
INFO:root:Epoch[243] Validation-accuracy=0.850000
INFO:root:Epoch[244] Batch [64]	Speed: 33422.11 samples/sec	accuracy=0.891346
INFO:root:Epoch[244] Batch [128]	Speed: 27170.35 samples/sec	accuracy=0.886230
INFO:root:Epoch[244] Batch [192]	Speed: 27561.57 samples/sec	accuracy=0.874512
INFO:root:Epoch[244] Train-accuracy=0.887601
INFO:root:Epoch[244] Time cost=0.475
INFO:root:Epoch[244] Validation-accuracy=0.850000
INFO:root:Epoch[245] Batch [64]	Speed: 35582.94 sampl

INFO:root:Epoch[264] Batch [192]	Speed: 39724.35 samples/sec	accuracy=0.875000
INFO:root:Epoch[264] Train-accuracy=0.888609
INFO:root:Epoch[264] Time cost=0.353
INFO:root:Epoch[264] Validation-accuracy=0.849583
INFO:root:Update[59401]: Change learning rate to 2.78128e-08
INFO:root:Epoch[265] Batch [64]	Speed: 37699.71 samples/sec	accuracy=0.891346
INFO:root:Epoch[265] Batch [128]	Speed: 43041.89 samples/sec	accuracy=0.886230
INFO:root:Epoch[265] Batch [192]	Speed: 43807.31 samples/sec	accuracy=0.875000
INFO:root:Epoch[265] Train-accuracy=0.888609
INFO:root:Epoch[265] Time cost=0.351
INFO:root:Epoch[265] Validation-accuracy=0.849583
INFO:root:Epoch[266] Batch [64]	Speed: 40841.63 samples/sec	accuracy=0.891346
INFO:root:Epoch[266] Batch [128]	Speed: 42520.96 samples/sec	accuracy=0.886230
INFO:root:Epoch[266] Batch [192]	Speed: 43103.23 samples/sec	accuracy=0.875000
INFO:root:Epoch[266] Train-accuracy=0.888609
INFO:root:Epoch[266] Time cost=0.346
INFO:root:Epoch[266] Validation-accuracy=0

INFO:root:Epoch[286] Batch [128]	Speed: 42573.01 samples/sec	accuracy=0.886230
INFO:root:Epoch[286] Batch [192]	Speed: 41848.84 samples/sec	accuracy=0.875732
INFO:root:Epoch[286] Train-accuracy=0.888609
INFO:root:Epoch[286] Time cost=0.356
INFO:root:Epoch[286] Validation-accuracy=0.849792
INFO:root:Epoch[287] Batch [64]	Speed: 39614.89 samples/sec	accuracy=0.891587
INFO:root:Epoch[287] Batch [128]	Speed: 43612.03 samples/sec	accuracy=0.886230
INFO:root:Epoch[287] Batch [192]	Speed: 43985.64 samples/sec	accuracy=0.875732
INFO:root:Epoch[287] Train-accuracy=0.888609
INFO:root:Epoch[287] Time cost=0.342
INFO:root:Epoch[287] Validation-accuracy=0.849792
INFO:root:Epoch[288] Batch [64]	Speed: 25152.11 samples/sec	accuracy=0.891587
INFO:root:Epoch[288] Batch [128]	Speed: 27507.16 samples/sec	accuracy=0.886230
INFO:root:Epoch[288] Batch [192]	Speed: 40769.04 samples/sec	accuracy=0.875732
INFO:root:Epoch[288] Train-accuracy=0.888609
INFO:root:Epoch[288] Time cost=0.462
INFO:root:Epoch[288] Val

Accuracy: 84.97916666666666 %


In [16]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DNN_SVM'
epoch = 300
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DNN_SVM-0300.params"


In [17]:
modelMetric(mod, 'Finish predict of DNN+SVM...')

INFO:root:Finish predict of DNN+SVM...
INFO:root:Test accuracy = 0.849519


             precision    recall  f1-score   support

        0.0       0.88      0.84      0.86      2666
        1.0       0.81      0.86      0.83      2112

avg / total       0.85      0.85      0.85      4778

[[2242  424]
 [ 295 1817]]


## 2.3 Deeper DNN with dropout + SVM

In [18]:
# Deeper DNN with dropout + SVM
data = mx.symbol.Variable('data')
data = mx.sym.Flatten(data=data)

fc1  = mx.symbol.FullyConnected(data = data, name='fc1', num_hidden=1024)
act1 = mx.symbol.Activation(data = fc1, name='relu1', act_type="relu")
fc2  = mx.symbol.FullyConnected(data = act1, name = 'fc2', num_hidden = 1024)
act2 = mx.symbol.Activation(data = fc2, name='relu2', act_type="relu")
drop1 = mx.sym.Dropout(data=act2, p=0.5)

fc3  = mx.symbol.FullyConnected(data = drop1, name = 'fc3', num_hidden = 512)
act3 = mx.symbol.Activation(data = fc3, name='relu3', act_type="relu")
fc4  = mx.symbol.FullyConnected(data = act3, name = 'fc4', num_hidden = 512)
act4 = mx.symbol.Activation(data = fc4, name='relu4', act_type="relu")

drop2 = mx.sym.Dropout(data=act4, p=0.5)

fc5  = mx.symbol.FullyConnected(data = drop2, name='fc5', num_hidden=2)

ddnn = mx.symbol.SVMOutput(data=fc5, name='svm')

# draw network chart
shape = {"data" : (batch_size, 3, 32, 32)}
mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ddnn, shape=shape)
network_chart.format = 'png'
network_chart.render('DDNN + SVM')

'DDNN + SVM.png'

<img src='DDNN + SVM.png'>

In [19]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ddnn,         # Use the network we just defined
    label_names = ['svm_label'],
)
lr_sch = mx.lr_scheduler.FactorScheduler(step=1800, factor=0.9)

mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size), 
    num_epoch = 300,       # Train for 100 epochs
    optimizer = 'Adam',
    optimizer_params = {
        'learning_rate': 3e-7, # Learning rate
        'beta1': 0.9,       # Beta 1
        'beta2': 0.999,         # Beta 2
        'lr_scheduler': lr_sch #learning rate decrease schedule
    },
)
print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Batch [64]	Speed: 16949.63 samples/sec	accuracy=0.540625
INFO:root:Epoch[0] Batch [128]	Speed: 17936.60 samples/sec	accuracy=0.578857
INFO:root:Epoch[0] Batch [192]	Speed: 17325.58 samples/sec	accuracy=0.558838
INFO:root:Epoch[0] Train-accuracy=0.568044
INFO:root:Epoch[0] Time cost=0.865
INFO:root:Epoch[0] Validation-accuracy=0.557500
INFO:root:Epoch[1] Batch [64]	Speed: 17590.79 samples/sec	accuracy=0.562740
INFO:root:Epoch[1] Batch [128]	Speed: 17095.98 samples/sec	accuracy=0.558105
INFO:root:Epoch[1] Batch [192]	Speed: 17662.48 samples/sec	accuracy=0.546387
INFO:root:Epoch[1] Train-accuracy=0.551915
INFO:root:Epoch[1] Time cost=0.841
INFO:root:Epoch[1] Validation-accuracy=0.557500
INFO:root:Epoch[2] Batch [64]	Speed: 16522.50 samples/sec	accuracy=0.552404
INFO:root:Epoch[2] Batch [128]	Speed: 14765.29 samples/sec	accuracy=0.551758
INFO:root:Epoch[2] Batch [192]	Speed: 14040.27 samples/sec	accuracy=0.542969
INFO:root:Epoch[2] Train-accuracy=0.549395
INFO:root:Epoch

INFO:root:Epoch[22] Train-accuracy=0.821573
INFO:root:Epoch[22] Time cost=0.867
INFO:root:Epoch[22] Validation-accuracy=0.800833
INFO:root:Epoch[23] Batch [64]	Speed: 17190.99 samples/sec	accuracy=0.808654
INFO:root:Epoch[23] Batch [128]	Speed: 17825.97 samples/sec	accuracy=0.807373
INFO:root:Epoch[23] Batch [192]	Speed: 16483.98 samples/sec	accuracy=0.798340
INFO:root:Epoch[23] Train-accuracy=0.822077
INFO:root:Epoch[23] Time cost=0.838
INFO:root:Epoch[23] Validation-accuracy=0.799167
INFO:root:Update[5401]: Change learning rate to 2.18700e-07
INFO:root:Epoch[24] Batch [64]	Speed: 17097.78 samples/sec	accuracy=0.805529
INFO:root:Epoch[24] Batch [128]	Speed: 16059.73 samples/sec	accuracy=0.805176
INFO:root:Epoch[24] Batch [192]	Speed: 13991.51 samples/sec	accuracy=0.794922
INFO:root:Epoch[24] Train-accuracy=0.812500
INFO:root:Epoch[24] Time cost=0.934
INFO:root:Epoch[24] Validation-accuracy=0.796042
INFO:root:Epoch[25] Batch [64]	Speed: 15644.23 samples/sec	accuracy=0.805048
INFO:root:

INFO:root:Epoch[44] Validation-accuracy=0.823125
INFO:root:Epoch[45] Batch [64]	Speed: 16489.11 samples/sec	accuracy=0.831731
INFO:root:Epoch[45] Batch [128]	Speed: 16899.66 samples/sec	accuracy=0.822510
INFO:root:Epoch[45] Batch [192]	Speed: 17803.03 samples/sec	accuracy=0.819580
INFO:root:Epoch[45] Train-accuracy=0.835685
INFO:root:Epoch[45] Time cost=0.840
INFO:root:Epoch[45] Validation-accuracy=0.825000
INFO:root:Epoch[46] Batch [64]	Speed: 17431.90 samples/sec	accuracy=0.829808
INFO:root:Epoch[46] Batch [128]	Speed: 18027.14 samples/sec	accuracy=0.822266
INFO:root:Epoch[46] Batch [192]	Speed: 17721.62 samples/sec	accuracy=0.823486
INFO:root:Epoch[46] Train-accuracy=0.832157
INFO:root:Epoch[46] Time cost=0.810
INFO:root:Epoch[46] Validation-accuracy=0.825417
INFO:root:Epoch[47] Batch [64]	Speed: 16907.72 samples/sec	accuracy=0.831971
INFO:root:Epoch[47] Batch [128]	Speed: 17113.98 samples/sec	accuracy=0.828125
INFO:root:Epoch[47] Batch [192]	Speed: 17299.76 samples/sec	accuracy=0.8

INFO:root:Epoch[89] Batch [64]	Speed: 12917.11 samples/sec	accuracy=0.853365
INFO:root:Epoch[89] Batch [128]	Speed: 13646.13 samples/sec	accuracy=0.848389
INFO:root:Epoch[89] Batch [192]	Speed: 15322.94 samples/sec	accuracy=0.840576
INFO:root:Epoch[89] Train-accuracy=0.853831
INFO:root:Epoch[89] Time cost=1.048
INFO:root:Epoch[89] Validation-accuracy=0.839375
INFO:root:Epoch[90] Batch [64]	Speed: 16785.38 samples/sec	accuracy=0.854327
INFO:root:Epoch[90] Batch [128]	Speed: 17149.75 samples/sec	accuracy=0.848389
INFO:root:Epoch[90] Batch [192]	Speed: 17136.48 samples/sec	accuracy=0.842285
INFO:root:Epoch[90] Train-accuracy=0.850302
INFO:root:Epoch[90] Time cost=0.873
INFO:root:Epoch[90] Validation-accuracy=0.840208
INFO:root:Epoch[91] Batch [64]	Speed: 17595.22 samples/sec	accuracy=0.854567
INFO:root:Epoch[91] Batch [128]	Speed: 15486.76 samples/sec	accuracy=0.847168
INFO:root:Epoch[91] Batch [192]	Speed: 13995.35 samples/sec	accuracy=0.839355
INFO:root:Epoch[91] Train-accuracy=0.856351

INFO:root:Epoch[111] Batch [128]	Speed: 17496.22 samples/sec	accuracy=0.852539
INFO:root:Epoch[111] Batch [192]	Speed: 13912.32 samples/sec	accuracy=0.845215
INFO:root:Epoch[111] Train-accuracy=0.859879
INFO:root:Epoch[111] Time cost=0.991
INFO:root:Epoch[111] Validation-accuracy=0.843542
INFO:root:Epoch[112] Batch [64]	Speed: 17293.02 samples/sec	accuracy=0.861058
INFO:root:Update[25201]: Change learning rate to 6.86304e-08
INFO:root:Epoch[112] Batch [128]	Speed: 17293.94 samples/sec	accuracy=0.854004
INFO:root:Epoch[112] Batch [192]	Speed: 17433.44 samples/sec	accuracy=0.844238
INFO:root:Epoch[112] Train-accuracy=0.861391
INFO:root:Epoch[112] Time cost=0.829
INFO:root:Epoch[112] Validation-accuracy=0.843542
INFO:root:Epoch[113] Batch [64]	Speed: 14211.73 samples/sec	accuracy=0.859135
INFO:root:Epoch[113] Batch [128]	Speed: 12655.42 samples/sec	accuracy=0.854492
INFO:root:Epoch[113] Batch [192]	Speed: 17556.42 samples/sec	accuracy=0.843018
INFO:root:Epoch[113] Train-accuracy=0.857359


...

INFO:root:Epoch[220] Batch [192]	Speed: 16190.04 samples/sec	accuracy=0.852783
INFO:root:Epoch[220] Train-accuracy=0.865927
INFO:root:Epoch[220] Time cost=0.956
INFO:root:Epoch[220] Validation-accuracy=0.850000
INFO:root:Epoch[221] Batch [64]	Speed: 17399.57 samples/sec	accuracy=0.866106
INFO:root:Epoch[221] Batch [128]	Speed: 17259.83 samples/sec	accuracy=0.861572
INFO:root:Epoch[221] Batch [192]	Speed: 17611.83 samples/sec	accuracy=0.852295
INFO:root:Epoch[221] Train-accuracy=0.868448
INFO:root:Epoch[221] Time cost=0.832
INFO:root:Epoch[221] Validation-accuracy=0.850000
INFO:root:Epoch[222] Batch [64]	Speed: 18390.04 samples/sec	accuracy=0.865385
INFO:root:Epoch[222] Batch [128]	Speed: 17573.53 samples/sec	accuracy=0.864014
INFO:root:Epoch[222] Batch [192]	Speed: 16334.57 samples/sec	accuracy=0.849854
INFO:root:Epoch[222] Train-accuracy=0.868448
INFO:root:Epoch[222] Time cost=0.831
INFO:root:Epoch[222] Validation-accuracy=0.850000
INFO:root:Epoch[223] Batch [64]	Speed: 14467.33 sampl

INFO:root:Epoch[242] Batch [128]	Speed: 15178.20 samples/sec	accuracy=0.864502
INFO:root:Epoch[242] Batch [192]	Speed: 13996.21 samples/sec	accuracy=0.852051
INFO:root:Epoch[242] Train-accuracy=0.866935
INFO:root:Epoch[242] Time cost=0.958
INFO:root:Epoch[242] Validation-accuracy=0.850208
INFO:root:Epoch[243] Batch [64]	Speed: 16524.73 samples/sec	accuracy=0.864904
INFO:root:Epoch[243] Batch [128]	Speed: 14301.33 samples/sec	accuracy=0.863525
INFO:root:Epoch[243] Batch [192]	Speed: 13236.61 samples/sec	accuracy=0.853027
INFO:root:Epoch[243] Train-accuracy=0.868952
INFO:root:Epoch[243] Time cost=1.007
INFO:root:Epoch[243] Validation-accuracy=0.850208
INFO:root:Epoch[244] Batch [64]	Speed: 16895.68 samples/sec	accuracy=0.867067
INFO:root:Epoch[244] Batch [128]	Speed: 16838.83 samples/sec	accuracy=0.864990
INFO:root:Epoch[244] Batch [192]	Speed: 17529.62 samples/sec	accuracy=0.851318
INFO:root:Epoch[244] Train-accuracy=0.863911
INFO:root:Epoch[244] Time cost=0.839
INFO:root:Epoch[244] Val

INFO:root:Epoch[264] Batch [128]	Speed: 16720.20 samples/sec	accuracy=0.865967
INFO:root:Epoch[264] Batch [192]	Speed: 17356.87 samples/sec	accuracy=0.849365
INFO:root:Epoch[264] Train-accuracy=0.869960
INFO:root:Epoch[264] Time cost=0.927
INFO:root:Epoch[264] Validation-accuracy=0.850208
INFO:root:Update[59401]: now learning rate arrived at 1.00000e-08, will not change in the future
INFO:root:Epoch[265] Batch [64]	Speed: 17737.12 samples/sec	accuracy=0.868269
INFO:root:Epoch[265] Batch [128]	Speed: 17915.37 samples/sec	accuracy=0.863770
INFO:root:Epoch[265] Batch [192]	Speed: 17619.13 samples/sec	accuracy=0.851318
INFO:root:Epoch[265] Train-accuracy=0.870968
INFO:root:Epoch[265] Time cost=0.812
INFO:root:Epoch[265] Validation-accuracy=0.850208
INFO:root:Epoch[266] Batch [64]	Speed: 17357.56 samples/sec	accuracy=0.867067
INFO:root:Epoch[266] Batch [128]	Speed: 17803.29 samples/sec	accuracy=0.865479
INFO:root:Epoch[266] Batch [192]	Speed: 17609.56 samples/sec	accuracy=0.850586
INFO:root

INFO:root:Epoch[285] Time cost=0.831
INFO:root:Epoch[285] Validation-accuracy=0.850417
INFO:root:Epoch[286] Batch [64]	Speed: 17547.02 samples/sec	accuracy=0.865385
INFO:root:Epoch[286] Batch [128]	Speed: 17621.57 samples/sec	accuracy=0.862305
INFO:root:Epoch[286] Batch [192]	Speed: 13942.65 samples/sec	accuracy=0.852295
INFO:root:Epoch[286] Train-accuracy=0.870464
INFO:root:Epoch[286] Time cost=0.911
INFO:root:Epoch[286] Validation-accuracy=0.850417
INFO:root:Epoch[287] Batch [64]	Speed: 12486.20 samples/sec	accuracy=0.869471
INFO:root:Epoch[287] Batch [128]	Speed: 12997.96 samples/sec	accuracy=0.863037
INFO:root:Epoch[287] Batch [192]	Speed: 13019.80 samples/sec	accuracy=0.851074
INFO:root:Epoch[287] Train-accuracy=0.863911
INFO:root:Epoch[287] Time cost=1.086
INFO:root:Epoch[287] Validation-accuracy=0.850417
INFO:root:Epoch[288] Batch [64]	Speed: 16692.30 samples/sec	accuracy=0.867308
INFO:root:Epoch[288] Batch [128]	Speed: 17347.71 samples/sec	accuracy=0.862549
INFO:root:Epoch[288]

Accuracy: 85.0625 %


In [20]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'DDNN_SVM'
epoch = 300
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "DDNN_SVM-0300.params"


In [21]:
modelMetric(mod, 'Finish predict of DDNN + SVM...')

INFO:root:Finish predict of DDNN + SVM...
INFO:root:Test accuracy = 0.850356


             precision    recall  f1-score   support

        0.0       0.89      0.83      0.86      2666
        1.0       0.81      0.87      0.84      2112

avg / total       0.85      0.85      0.85      4778

[[2225  441]
 [ 274 1838]]


## 2.4 Inception Batach Normalization

In [22]:
# Reference: www.mxnet.io
# Basic Conv + BN + ReLU factory
def ConvFactory(data, num_filter, kernel, stride=(1,1), pad=(0, 0), act_type="relu"):
    conv = mx.symbol.Convolution(data=data, workspace=256,
                                 num_filter=num_filter, kernel=kernel, stride=stride, pad=pad)
    bn = mx.symbol.BatchNorm(data=conv)
    act = mx.symbol.Activation(data = bn, act_type=act_type)
    return act

# A Simple Downsampling Factory
def DownsampleFactory(data, ch_3x3):
    conv = ConvFactory(data=data, kernel=(3, 3), stride=(2, 2), num_filter=ch_3x3, pad=(1, 1))
    pool = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(2, 2),pad=(1,1), pool_type='max')
    concat = mx.symbol.Concat(*[conv, pool])
    return concat

# A Simple module
def SimpleFactory(data, ch_1x1, ch_3x3):
    conv1x1 = ConvFactory(data=data, kernel=(1, 1), pad=(0, 0), num_filter=ch_1x1)
    conv3x3 = ConvFactory(data=data, kernel=(3, 3), pad=(1, 1), num_filter=ch_3x3)
    concat = mx.symbol.Concat(*[conv1x1, conv3x3])
    return concat

def InceptionFactory(data, num_1x1, num_3x3red, num_3x3, num_d5x5red, num_d5x5, pool, proj, name):
    c1x1 = ConvFactoryB(data=data, num_filter=num_1x1, kernel=(1, 1), name=('%s_1x1' % name))
    c3x3r = ConvFactoryB(data=data, num_filter=num_3x3red, kernel=(1, 1), name=('%s_3x3' % name), suffix='_reduce')
    c3x3 = ConvFactoryB(data=c3x3r, num_filter=num_3x3, kernel=(3, 3), pad=(1, 1), name=('%s_3x3' % name))
    cd5x5r = ConvFactoryB(data=data, num_filter=num_d5x5red, kernel=(1, 1), name=('%s_double_3x3' % name), suffix='_reduce')
    cd5x5 = ConvFactoryB(data=cd5x5r, num_filter=num_d5x5, kernel=(5, 5), pad=(2, 2), name=('%s_double_3x3_1' % name))
    pooling = mx.symbol.Pooling(data=data, kernel=(3, 3), stride=(1, 1), pad=(1, 1), pool_type=pool, name=('%s_pool_%s_pool' % (pool, name)))
    cproj = ConvFactoryB(data=pooling, num_filter=proj, kernel=(1, 1), name=('%s_proj' %  name))
    concat = mx.symbol.Concat(*[c1x1, c3x3, cd5x5, cproj], name='ch_concat_%s_chconcat' % name)
    return concat

In [25]:
# Inception Batch Normalization Network
#https://arxiv.org/pdf/1502.03167.pdf
data = mx.sym.Variable('data')

conv1 = ConvFactory(data=data, kernel=(3,3), pad=(1,1), num_filter=96, act_type="relu")
in3a = SimpleFactory(conv1, 32, 32)
in3b = SimpleFactory(in3a, 32, 48)
in3c = DownsampleFactory(in3b, 80)
in4a = SimpleFactory(in3c, 112, 48)
in4b = SimpleFactory(in4a, 96, 64)
in4c = SimpleFactory(in4b, 80, 80)
in4d = SimpleFactory(in4c, 48, 96)
in4e = DownsampleFactory(in4d, 96)
in5a = SimpleFactory(in4e, 176, 160)
in5b = SimpleFactory(in5a, 176, 160)
pool = mx.symbol.Pooling(data=in5b, pool_type="avg", kernel=(7,7), name="global_pool")
flatten = mx.symbol.Flatten(data=pool, name="flatten1")
fc0  = mx.symbol.FullyConnected(data = flatten, name = 'fc0', num_hidden = 10)
act0 = mx.symbol.Activation(data = fc0, name='relu0', act_type="relu")
fc = mx.symbol.FullyConnected(data=act0, num_hidden=2, name="fc1")
ibn = mx.symbol.SVMOutput(data=fc, name='svm')

# draw network chart
shape = {"data" : (batch_size, 3, 32, 32)}
mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart=mx.viz.plot_network(symbol=ibn, shape=shape)
network_chart.format = 'png'
network_chart.render('Inception Batch Normalization')

'Inception Batch Normalization.png'

<img src = 'Inception Batch Normalization.png'>

In [27]:
mod = mx.mod.Module(
    context = mx.gpu(0),  # Run on GPU 0
    symbol = ibn,         # Use the network we just defined
    label_names = ['svm_label'],
)

mod.fit(
    train_data=train_iters,
    eval_data=val_iters,  
    batch_end_callback = mx.callback.Speedometer(batch_size, batch_size),  
    num_epoch = 50,       # Train for 100 epochs
    optimizer_params = {
        'learning_rate': 1e-4,  # Learning rate
        'momentum': 0.9,       # Momentum for SGD with momentum
        'wd': 0.00001,         # Weight decay for regularization
    },
)

print('Accuracy:', mod.score(val_iters, mx.metric.Accuracy())[0][1]*100, '%')

INFO:root:Epoch[0] Train-accuracy=0.533750
INFO:root:Epoch[0] Time cost=1.384
INFO:root:Epoch[0] Validation-accuracy=0.557500
INFO:root:Epoch[1] Batch [64]	Speed: 1192.83 samples/sec	accuracy=0.550481
INFO:root:Epoch[1] Batch [128]	Speed: 1196.15 samples/sec	accuracy=0.551270
INFO:root:Epoch[1] Batch [192]	Speed: 1197.79 samples/sec	accuracy=0.542236
INFO:root:Epoch[1] Train-accuracy=0.549395
INFO:root:Epoch[1] Time cost=11.962
INFO:root:Epoch[1] Validation-accuracy=0.557500
INFO:root:Epoch[2] Batch [64]	Speed: 1200.80 samples/sec	accuracy=0.550240
INFO:root:Epoch[2] Batch [128]	Speed: 1200.08 samples/sec	accuracy=0.550293
INFO:root:Epoch[2] Batch [192]	Speed: 1189.47 samples/sec	accuracy=0.597168
INFO:root:Epoch[2] Train-accuracy=0.627520
INFO:root:Epoch[2] Time cost=11.953
INFO:root:Epoch[2] Validation-accuracy=0.619583
INFO:root:Epoch[3] Batch [64]	Speed: 1191.07 samples/sec	accuracy=0.627404
INFO:root:Epoch[3] Batch [128]	Speed: 1186.66 samples/sec	accuracy=0.614746
INFO:root:Epoch

INFO:root:Epoch[23] Train-accuracy=0.991431
INFO:root:Epoch[23] Time cost=12.273
INFO:root:Epoch[23] Validation-accuracy=0.798542
INFO:root:Epoch[24] Batch [64]	Speed: 1144.47 samples/sec	accuracy=0.994231
INFO:root:Epoch[24] Batch [128]	Speed: 1137.27 samples/sec	accuracy=0.994629
INFO:root:Epoch[24] Batch [192]	Speed: 1145.40 samples/sec	accuracy=0.994873
INFO:root:Epoch[24] Train-accuracy=0.991935
INFO:root:Epoch[24] Time cost=12.561
INFO:root:Epoch[24] Validation-accuracy=0.772500
INFO:root:Epoch[25] Batch [64]	Speed: 1157.75 samples/sec	accuracy=0.994231
INFO:root:Epoch[25] Batch [128]	Speed: 1156.50 samples/sec	accuracy=0.995117
INFO:root:Epoch[25] Batch [192]	Speed: 1152.17 samples/sec	accuracy=0.995117
INFO:root:Epoch[25] Train-accuracy=0.992440
INFO:root:Epoch[25] Time cost=12.402
INFO:root:Epoch[25] Validation-accuracy=0.806667
INFO:root:Epoch[26] Batch [64]	Speed: 1185.41 samples/sec	accuracy=0.994471
INFO:root:Epoch[26] Batch [128]	Speed: 1125.40 samples/sec	accuracy=0.9958

INFO:root:Epoch[46] Batch [192]	Speed: 1174.43 samples/sec	accuracy=0.999512
INFO:root:Epoch[46] Train-accuracy=1.000000
INFO:root:Epoch[46] Time cost=12.143
INFO:root:Epoch[46] Validation-accuracy=0.805417
INFO:root:Epoch[47] Batch [64]	Speed: 1185.30 samples/sec	accuracy=0.999038
INFO:root:Epoch[47] Batch [128]	Speed: 1157.35 samples/sec	accuracy=0.999512
INFO:root:Epoch[47] Batch [192]	Speed: 1160.80 samples/sec	accuracy=0.999512
INFO:root:Epoch[47] Train-accuracy=1.000000
INFO:root:Epoch[47] Time cost=12.226
INFO:root:Epoch[47] Validation-accuracy=0.802917
INFO:root:Epoch[48] Batch [64]	Speed: 1147.53 samples/sec	accuracy=0.999760
INFO:root:Epoch[48] Batch [128]	Speed: 1153.97 samples/sec	accuracy=0.999756
INFO:root:Epoch[48] Batch [192]	Speed: 1158.52 samples/sec	accuracy=0.999268
INFO:root:Epoch[48] Train-accuracy=1.000000
INFO:root:Epoch[48] Time cost=12.396
INFO:root:Epoch[48] Validation-accuracy=0.804167
INFO:root:Epoch[49] Batch [64]	Speed: 1172.66 samples/sec	accuracy=0.9997

Accuracy: 80.625 %


In [28]:
# save a model to mymodel-symbol.json and mymodel-0100.params
prefix = 'Inception_BN'
epoch = 50
mod.save_checkpoint(prefix, epoch)

INFO:root:Saved checkpoint to "Inception_BN-0050.params"


In [29]:
modelMetric(mod, 'Finish predict of Inception BN..')

INFO:root:Finish predict of Inception BN..
INFO:root:Test accuracy = 0.806195


             precision    recall  f1-score   support

        0.0       0.86      0.78      0.82      2666
        1.0       0.75      0.84      0.79      2112

avg / total       0.81      0.81      0.81      4778

[[2081  585]
 [ 341 1771]]
